# Simple neural network

## Prepare colab

In [1]:
!pip install pydot
!pip install GraphViz

In [2]:
! ls

gdrive	log.txt  sample_data


## Config

In [0]:
### CONFIG ###
thefile = 'prepared-20181001_prefix-pNN_input_2tag_Sig_BKGs.pickle'
### END ###

do3tag = False
doTrainNetwork = True

layer_size = 1
dropout = None
nepochs = 50
batch_size = 128
modeltag = 'nn_l{0}e{1}b{2}'.format(layer_size,nepochs,batch_size)

netfile = "model_{0}.h5".format(modeltag)
graphfile = "graph_{0}.png".format(modeltag)
file_performance = "perf_{0}.pickle".format(modeltag)
file_minmax_scaler = "minmax_scaler.pickle"
file_quantile_scaler = "quantile_scaler.pickle"

## Import

In [4]:
import argparse
import os
import re
from glob import glob
import numpy as np
import pandas as pd
import sys

import pickle

from sklearn.utils import shuffle
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.externals import joblib

from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout
from keras.regularizers import l1_l2
from keras.optimizers import SGD
from keras import backend as K
from keras.utils import plot_model

Using TensorFlow backend.


## Assistant functions

In [0]:
import logging
import sys

#############################################################
def setup_custom_logger(name):
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')
    handler = logging.FileHandler('log.txt', mode='w')
    handler.setFormatter(formatter)
    screen_handler = logging.StreamHandler(stream=sys.stdout)
    screen_handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(handler)
    logger.addHandler(screen_handler)
    return logger

#############################################################
def get_model(n_invars, n_hidden_nodes, regularization=None, dropout=None):
    if not isinstance(n_hidden_nodes, list):
        n_hidden_nodes = [n_hidden_nodes]

    x = Input(shape=(n_invars,))
    d = x
    for n in n_hidden_nodes:
        d = Dense(n, activation="relu", kernel_regularizer=regularization)(d)
        if dropout:
            d = Dropout(dropout)(d)
    y = Dense(1, activation="sigmoid")(d)
    return Model(x, y)
##########################################################
def preprocess(X, quantile_trsf=None, minmax_trsf=None):
    if quantile_trsf is None and minmax_trsf is None:
        quantile_trsf = QuantileTransformer()
        minmax_trsf = MinMaxScaler()
        quantile_trsf.fit(X[:, :-1])
        minmax_trsf.fit(X[:, -1][:, np.newaxis])
    elif quantile_trsf is not None and minmax_trsf is not None:
        pass
    else:
        return None
    X_trsf_no_mass = quantile_trsf.transform(X[:, :-1])
    X_trsf_mass = minmax_trsf.transform(X[:, -1][:, np.newaxis])

    return quantile_trsf, minmax_trsf, np.hstack([X_trsf_no_mass, X_trsf_mass])
##########################################################


#############################################################
import numpy as np

# map mA,mH -> DISD
dict_mAmH_dsid = {
  'ggF_llbb' : {
    (230,130) : 306939,
    (250,130) : 306940,
    (230,150) : 306941,
    (300,130) : 306942,
    (300,150) : 306943,
    (300,200) : 306944,
    (350,250) : 306945,
    (400,130) : 306946,
    (400,200) : 306948,
    (400,250) : 344587,
    (500,130) : 306952,
    (500,200) : 306955,
    (500,300) : 306958,
    (500,350) : 308468,
    (500,400) : 306959,
    (600,130) : 306962,
    (600,300) : 306966,
    (600,400) : 344588,
    (600,450) : 308469,
    (600,500) : 306967,
    (700,130) : 306968,
    (700,200) : 306970,
    (700,300) : 306972,
    (700,400) : 306973,
    (700,500) : 306974,
    (700,600) : 308568,
    (800,130) : 308569,
    (800,300) : 308570,
    (800,500) : 344589,
    (800,700) : 308571,
  },
}

mH_min = 130
mH_max = 700
mA_min = 230
mA_max = 800

# mAmH -> DSID list
#############################################################
# mAmH_list is a list of mA,mH pair
# prod: ggF_llbb, bbA_llbb ...
def mAmH2disd( mAmH_list, prod ):
  return [ dict_mAmH_dsid[prod][(mA,mH)] for mA,mH in mAmH_list ]

# random sampling of background mA and mH
#############################################################
# nm: mA or mH
# the whole df
def sample_bkg_mass(nm, df, method=0, seed=None):
  n_bkg = (~df.IsSignal).sum()
  m_bkg = None
  if seed:
    np.random.seed(seed)
  if method==0:
    m_bkg = np.random.choice(df[nm][~df.IsSignal],size=n_bkg)
  else:
    #m_bkg = np.random.uniform(np.min(m[isSig]),np.max(m[isSig]),size=n_bkg)
    if nm == 'mA':
      m_bkg = np.random.uniform( mA_min, mH_min, size=n_bkg)
    if nm == 'mH':
      m_bkg = np.random.uniform( mH_min, mA_max, size=n_bkg)
  df.loc[~df.IsSignal, nm] = m_bkg

# choose signal samples
#############################################################
# mAmH_list is a list of mA,mH pair
# prod: ggF_llbb, bbA_llbb ...
def choose_signal_samples( df, mAmH_list, prod ):
  print('choose_signal_samples: signal tot # =',df.IsSignal.sum())
  dsid_list = mAmH2disd( mAmH_list, prod )
  df_chosen = df[ ((df.IsSignal) & (df.label.isin(dsid_list)) ) | (~df.IsSignal) ]
  print('choose_signal_samples: signal tot chosen to use # =',df_chosen.IsSignal.sum() )
  return df_chosen

# list input variables
#############################################################
def input_variables( do3tag, df ):
  inputs = None
  if do3tag:
    inputs = df[["lep0pt","lep0eta","lep0phi",
             "lep1pt","lep1eta","lep1phi",
             "jet0pt","jet0eta","jet0phi",
             "jet1pt","jet1eta","jet1phi",
             "jet2pt","jet2eta","jet2phi",
             "mA","mH"]].values
  else:
    inputs = df[["lep0pt","lep0eta","lep0phi",
             "lep1pt","lep1eta","lep1phi",
             "jet0pt","jet0eta","jet0phi",
             "jet1pt","jet1eta","jet1phi",
             "mA","mH"]].values
  return inputs


## System

In [0]:
# system
logger = setup_custom_logger('logger_algo_nn')

## Load in data

In [7]:
logger.info('1. Load in dataset ...')

if os.path.isfile('/content/gdrive/My Drive/AZHOptimisation/{0}'.format(thefile)):
  pass
else:
  logger.info(' + mount google drive ...')
  from google.colab import drive
  drive.mount('/content/gdrive')

2018-10-30 19:36:54 INFO     1. Load in dataset ...


In [8]:
logger.info(' + load in dataframe ...')
df = None
with open('/content/gdrive/My Drive/AZHOptimisation/{0}'.format(thefile), 'rb') as f:
  df = pickle.load( f )
  logger.info('load in done')
print(df.head())

2018-10-30 19:36:55 INFO      + load in dataframe ...
2018-10-30 19:37:18 INFO     load in done
   label    weight     lep0pt   lep0eta   lep0phi     lep1pt   lep1eta  \
0    0.0  1.127729  67.842850  0.202458  0.776767  24.886770 -0.710250   
1    0.0  0.981583  53.870251  1.400823 -0.405050  32.279022  0.701465   
2    0.0  1.276938  43.058762  0.653615 -2.372478  31.318354 -1.273499   
3    0.0 -1.263140  40.866436 -0.733332 -0.106703  31.655827 -1.992335   
4    0.0 -2.159261  82.256416  0.107891  1.170636  24.290808  0.053033   

    lep1phi     jet0pt   jet0eta    ...     jet2eta  jet2phi        MET  \
0 -2.265613  82.877945  1.044228    ...         0.0      0.0  20.968199   
1  2.193279  50.149277  0.604722    ...         0.0      0.0  30.534142   
2  2.797295  88.430099 -1.511340    ...         0.0      0.0  31.228664   
3  2.209157  49.726997  0.705296    ...         0.0      0.0   9.700199   
4 -2.224541  60.990715  2.497605    ...         0.0      0.0   5.708796   

     MET

## Prepare training testing dataset

In [9]:
logger.info('2. Prepare training testing datasets')
logger.info(' + choose signal samples (may not all mA,mH)')
df = choose_signal_samples( df, [ [400,200], [700,200], [700,500] ], 'ggF_llbb' )
logger.info(' + generate random mA,mH for bkg')
sample_bkg_mass('mA', df, 1) # mH
sample_bkg_mass('mH', df, 1) # mA
logger.info(' + connect input variables depending on #bjets')
X = input_variables( do3tag, df )
logger.info(' + prepare target Y with type int')
Y = df.IsSignal.values.astype(int)
logger.info(' + shuffle all chosen entries')
X, Y = shuffle(X,Y)
logger.info(' + split sample for training 2/3, testing 1/3')
N = len(X)
separ = int(2*N/3)
X_train = X[:separ]
Y_train  = Y[:separ]
X_test = X[separ:]
Y_test = Y[separ:]
logger.info('Training sample: {0}; Tesiting sample: {1}'.format( len(X_train), len(X_test) ))

2018-10-30 19:37:19 INFO     2. Prepare training testing datasets
2018-10-30 19:37:19 INFO      + choose signal samples (may not all mA,mH)
choose_signal_samples: signal tot # = 310511
choose_signal_samples: signal tot chosen to use # = 34282
2018-10-30 19:37:23 INFO      + generate random mA,mH for bkg
2018-10-30 19:37:25 INFO      + connect input variables depending on #bjets
2018-10-30 19:37:26 INFO      + prepare target Y with type int
2018-10-30 19:37:26 INFO      + shuffle all chosen entries
2018-10-30 19:37:30 INFO      + split sample for training 2/3, testing 1/3
2018-10-30 19:37:30 INFO     Training sample: 11004202; Tesiting sample: 5502102


## Train and test NN

In [0]:
model = None
minmax_trsf = None
quantile_trsf = None
fitres = []

if doTrainNetwork:
    logger.info('3. Train NN with {0} layers, {1} epochs, {2} batch size, {3} dropout. model tag is {4}'.format(layer_size,nepochs,batch_size,dropout,modeltag) )

    best_loss = 1e9
    min_delta = 0.0001
    patience = 25
    epochs_not_improved = 0
    # Reduce LR on plateau
    lr_best_loss = 1e9
    lr_patience = 10
    lr_factor = 0.5
    lr_epochs_not_improved = 0

    _, n_invars = X.shape
    model = get_model(n_invars, layer_size, dropout=dropout)
    sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd,loss="binary_crossentropy",metrics=["accuracy"])
    #plot_model(model, to_file=graphfile, show_shapes=True)
    # Preprocessing
    logger.info(' + preprocessing ...')
    quantile_trsf, minmax_trsf, X_train_trsf = preprocess(X_train)
    _,_, X_test_trsf = preprocess(X_test, quantile_trsf, minmax_trsf)

    logger.info(' + training ...')
    logger.info("{:>10}{:>10}".format("Epoch", "Loss"))
    _res = None
    for epoch in range(nepochs):
        # sample_bkg_mass(X_train_trsf[:, -1], Y_train) # <--------- TEST IT
        _res = model.fit(X_train_trsf, Y_train, batch_size=batch_size, epochs=1, validation_data=(X_test_trsf,Y_test), verbose=0)
        fitres.append(_res.history)
        loss_epoch = _res.history['loss'][0] # as nepoch is 1

        logger.info("{:>10.0f}{:>10.5f}".format(epoch, loss_epoch))
        # Reduce LR on plateau
        if loss_epoch + min_delta < lr_best_loss:
            lr_best_loss = loss_epoch
            lr_epochs_not_improved = 0
        else:
            lr_epochs_not_improved += 1

        # Early stopping
        if loss_epoch + min_delta < best_loss:
            best_loss = loss_epoch
            epochs_not_improved = 0
        else:
            epochs_not_improved += 1

        # Reduce LR on plateau
        if lr_epochs_not_improved > lr_patience:
            current_lr = K.get_value(model.optimizer.lr)
            logger.info("Current LR: {}".format(current_lr))
            logger.info("Reducing LR to: {}".format(lr_factor * current_lr))
            K.set_value(model.optimizer.lr, lr_factor * current_lr)
            lr_epochs_not_improved = 0

        # Early stopping
        if epochs_not_improved > patience:
            logger.info("Early stopping after epoch {}".format(epoch))
            break
    # loop ends here
    # now save the network
    logger.info(' + save history of performance to file {0}'.format(file_performance))
    with open( file_performance, 'wb') as savehandle:
        pickle.dump(fitres, savehandle, protocol=3)
    logger.info(' + dump model and transformers to files {0} {1} {2}'.format(netfile,file_minmax_scaler,file_quantile_scaler))
    joblib.dump(minmax_trsf, file_minmax_scaler)
    joblib.dump(quantile_trsf, file_quantile_scaler)
    model.save(netfile)

    # print perfomance
    logger.info(' + print performance')
    logger.info('loss:{0}'.format(fitres[-1]['loss']))
    logger.info('loss validation:{0}'.format(fitres[-1]['val_loss']))
    logger.info('accuracy:{0}'.format(fitres[-1]['acc']))
    logger.info('accuracy validation:{0}'.format(fitres[-1]['val_acc']))

else:
    logger.info( "3. No traininng , directly application, read in the nn model and transformer files")
    logger.info(' + read in files {0} {1} {2}'.format(netfile,file_minmax_scaler,file_quantile_scaler))
    model = load_model(netfile)
    quantile_scaler = joblib.load(file_quantile_scaler)
    minmax_scaler = joblib.load(file_minmax_scaler)

2018-10-30 19:38:14 INFO     3. Train NN with 1 layers, 50 epochs, 128 batch size, None dropout. model tag is nn_l1e50b128
2018-10-30 19:38:14 INFO      + preprocessing ...
2018-10-30 19:39:40 INFO      + training ...
2018-10-30 19:39:40 INFO          Epoch      Loss
2018-10-30 19:46:45 INFO              0   0.00618
2018-10-30 19:53:49 INFO              1   0.00448
2018-10-30 20:00:51 INFO              2   0.00398
2018-10-30 20:07:53 INFO              3   0.00371
2018-10-30 20:14:54 INFO              4   0.00346
2018-10-30 20:21:56 INFO              5   0.00330
2018-10-30 20:28:59 INFO              6   0.00317
2018-10-30 20:36:02 INFO              7   0.00303
2018-10-30 20:43:03 INFO              8   0.00291
2018-10-30 20:50:06 INFO              9   0.00282
2018-10-30 20:57:10 INFO             10   0.00276
